In [108]:
from ensemble_boxes import *
from glob import glob
import pathlib
import os

In [109]:
mode = "modern"

In [110]:

if mode == "modern":
    yolov8_result_path = pathlib.Path("/home/fummicc1/codes/signate/runs/detect/predict11")
    yolov5_result_path = pathlib.Path("/home/fummicc1/codes/signate/yolov5/runs/detect/exp10")
elif mode == "old":
    yolov8_result_path = pathlib.Path("/home/fummicc1/codes/signate/runs/detect/predict12")
    yolov5_result_path = pathlib.Path("/home/fummicc1/codes/signate/yolov5/runs/detect/exp11")
image_id_list = [file.stem for file in list(yolov5_result_path.iterdir()) if file.is_file()]

yolov8_result_path

PosixPath('/home/fummicc1/codes/signate/runs/detect/predict11')

In [111]:
def get_yolov5_label_path(id: str) -> pathlib.Path:
    return yolov5_result_path / "labels" / f"{id}.txt"

def get_yolov8_label_path(id: str) -> pathlib.Path:
    return yolov8_result_path / "labels" / f"{id}.txt"

def map_label_id_from_modern_to_all(modern_id: int) -> int:
    if mode == "old":
        return modern_id
    if modern_id == 0 or modern_id == 1:
        return modern_id
    return modern_id + 2

In [112]:
for v8file, v5file in zip(list(yolov8_result_path.iterdir()), list(yolov5_result_path.iterdir())):
    boxes_list = [[], []]

    scores_list = [[], []]
    labels_list = [[], []]
    weights = [1, 1] if mode == "modern" else [1, 1]

    iou_thr = 0.5
    skip_box_thr = 0.3 if mode == "modern" else 0.3
    
    if (not v5file.is_file()) or (not v8file.is_file()):
        continue
    if v8file.stem != v5file.stem:
        continue
    name = v8file.stem    
    
    v5ann_file = get_yolov5_label_path(name)
    if not v5ann_file.exists():
        continue
    v5lines = v5ann_file.read_text().split("\n")
    for line in v5lines:
        if line == "":
            continue
        class_id, x_min, y_min, width, height, conf = map(float, line.split())
        class_id = int(class_id)
        if class_id in [0, 7]:
            continue
        if class_id == 1:
            # v5の方が性能がよい
            overall_stat = x_min, y_min, width, height
        boxes_list[0].append([x_min, y_min, x_min + width, y_min + height])
        labels_list[0].append(class_id)
        scores_list[0].append(conf)
    v8ann_file = get_yolov8_label_path(name)
    if not v8ann_file.exists():
        continue
    v8lines = v8ann_file.read_text().split("\n")    
    for line in v8lines:
        if line == "":
            continue
        class_id, x_min, y_min, width, height, conf = map(float, line.split())
        boxes_list[1].append([x_min, y_min, x_min + width, y_min + height])
        labels_list[1].append(class_id)
        scores_list[1].append(conf)
        
    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    os.makedirs(f"/home/fummicc1/codes/signate/emsemble/{mode}-ret", exist_ok=True)
    out_path = pathlib.Path(f"/home/fummicc1/codes/signate/emsemble/{mode}-ret")
    out_path = out_path / f"{name}.txt"
    out_path.unlink(missing_ok=True)
    for box, score, label in zip(boxes, scores, labels):                
        with open(out_path, "a") as f:
            label = int(label)
            label = map_label_id_from_modern_to_all(label)
            if int(label) == 1:                
                f.write(f"{int(label)} {overall_stat[0]} {overall_stat[1]} {overall_stat[2]} {overall_stat[3]}")
            else:
                f.write(f"{int(label)} {box[0]} {box[1]} {box[2] - box[0]} {box[3] - box[1]}") 
            f.write("\n")
    

/home/fummicc1/codes/signate/venv/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:51: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/home/fummicc1/codes/signate/venv/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:63: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')


In [113]:
v5file, labels, boxes

(PosixPath('/home/fummicc1/codes/signate/yolov5/runs/detect/exp10/test_995415_0003.jpg'),
 array([1., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
        6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
        6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
        6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
        6., 6., 6.]),
 array([[0.48131588, 0.37260216, 1.        , 1.        ],
        [0.28545606, 0.27595788, 0.30549127, 0.34318808],
        [0.84465307, 0.40555072, 0.86466968, 0.44966263],
        [0.8771832 , 0.29430836, 0.89589411, 0.38155937],
        [0.2534973 , 0.27566007, 0.27315566, 0.34090573],
        [0.77877676, 0.28330511, 0.7986719 , 0.34968597],
        [0.74598122, 0.2938337 , 0.76486713, 0.38162991],
        [0.3847743 , 0.40873611, 0.40500218, 0.47424963],
        [0.41723374, 0.26302135, 0.43726736, 0.35339862],
        [0.38448316, 0.33194053, 0.40480173, 0.37494013],
    

In [114]:
np.array(labels_list)[0]

/tmp/ipykernel_4170425/856421453.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(labels_list)[0]


[6,
 6,
 6,
 6,
 1,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 1]

In [115]:
import numpy as np
import pandas as pd

df = pd.DataFrame(scores_list)
df

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,0.257373,0.257868,0.267561,0.270441,0.272569,0.283705,0.284984,0.298799,0.307344,0.329691,...,0.855762,0.858931,0.860347,0.86117,0.867262,0.868656,0.874885,0.879182,0.889473,0.928444
1,0.258796,0.266179,0.296058,0.297455,0.332889,0.339055,0.364168,0.371970,0.379710,0.391771,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
